In [74]:
import pandas as pd 
from sklearn.impute import SimpleImputer
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression


In [60]:
#preprocessing data
df = pd.read_csv('train_data.csv')

df.drop('Loan_ID',axis = 1, inplace = True)
name_columns = list(df.columns.values)
#checking nan values
imp = SimpleImputer(missing_values = np.nan, strategy = 'most_frequent')
imp.fit(df)
df = imp.transform(df)
# format all string to 1 vs 0

df = pd.DataFrame(df,columns= name_columns)
# apply one-hot encoding 
Y = df.iloc[:,-1].values 
y = [1 if e=='Y' else 0 for e in Y]
X = df.drop('Loan_Status', axis = 1)
data_new = pd.get_dummies(X,columns =['Gender','Married','Dependents','Education','Self_Employed','Property_Area'] )



In [93]:
#format data test
df_test = pd.read_csv('test_data.csv')
df_test.drop('Loan_ID',axis = 1, inplace = True)
name_columns_test = list(df_test.columns.values)
imp_test = SimpleImputer(missing_values = np.nan, strategy ='most_frequent')
imp_test.fit(df_test)
df_test = imp_test.transform(df_test)
df_test = pd.DataFrame(df_test, columns = name_columns_test)
df_test_new = pd.get_dummies(df_test, columns =['Gender','Married','Dependents','Education','Self_Employed','Property_Area'] )
df_test_new


,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Gender_Female,Gender_Male,Married_No,Married_Yes,Dependents_0,Dependents_1,Dependents_2,Dependents_3+,Education_Graduate,Education_Not Graduate,Self_Employed_No,Self_Employed_Yes,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban
0,5720,0,110.0,360.0,1.0,0,1,0,1,1,0,0,0,1,0,1,0,0,0,1
1,3076,1500,126.0,360.0,1.0,0,1,0,1,0,1,0,0,1,0,1,0,0,0,1
2,5000,1800,208.0,360.0,1.0,0,1,0,1,0,0,1,0,1,0,1,0,0,0,1
3,2340,2546,100.0,360.0,1.0,0,1,0,1,0,0,1,0,1,0,1,0,0,0,1
4,3276,0,78.0,360.0,1.0,0,1,1,0,1,0,0,0,0,1,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362,4009,1777,113.0,360.0,1.0,0,1,0,1,0,0,0,1,0,1,0,1,0,0,1
363,4158,709,115.0,360.0,1.0,0,1,0,1,1,0,0,0,1,0,1,0,0,0,1
364,3250,1993,126.0,360.0,1.0,0,1,1,0,1,0,0,0,1,0,1,0,0,1,0
365,5000,2393,158.0,360.0,1.0,0,1,0,1,1,0,0,0,1,0,1,0,1,0,0


In [98]:
if __name__ =="__main__":
    pipeline = Pipeline([
        ('clf', RandomForestClassifier(criterion = 'entropy'))
    ])
    parameters= {
        'clf__n_estimators': (5,10,20,50),
        'clf__max_depth': (50,150,250),
        'clf__min_samples_split': (1,2,3),
        'clf__min_samples_leaf': (1,2,3)
    }
    #get X-train y_train  
    # X_train, X_test, y_train, y_test = train_test_split(data_new,y)
    grid_search = GridSearchCV(pipeline, parameters,n_jobs = -1, verbose = 1, scoring ='f1')
    grid_search.fit(data_new, y)
    print('best score {}'.format(grid_search.best_score_))
    best_parameters = grid_search.best_estimator_.get_params()
    for i, value in enumerate(parameters.keys()):
        print("{}: {}".format(value, best_parameters[value]))


Fitting 5 folds for each of 108 candidates, totalling 540 fits
best score 0.8730784634280837
clf__n_estimators: 50
clf__max_depth: 50
clf__min_samples_split: 3
clf__min_samples_leaf: 3


In [99]:
if __name__ =='__main__':
    pipeline_Logistic = Pipeline([
        ('clf', LogisticRegression())
        ])
    parameteres_logistic ={
        'clf__C':(0.1,1,10)
    }
grid_logistic = GridSearchCV(pipeline_Logistic,parameteres_logistic,n_jobs =-1,verbose = 1, scoring = 'f1')
grid_logistic.fit(data_new, y)
print("best score: {}".format(grid_logistic.best_score_))
best_parameters_logistic = grid_logistic.best_estimator_.get_params()
for value in sorted(parameteres_logistic.keys()):
    print(value, best_parameters_logistic[value])

Fitting 5 folds for each of 3 candidates, totalling 15 fits
best score: 0.8728941830162196
clf__C 1


In [97]:
y_predict_tree = grid_search.predict(df_test_new)
y_predict_logistic = grid_logistic.predict(df_test_new)
for i, value in enumerate(y_predict_tree):
    print("tree :{} logistic{}".format(y_predict_tree[i], y_predict_logistic[i]))

tree :1 logistic1
tree :1 logistic1
tree :1 logistic1
tree :1 logistic1
tree :0 logistic1
tree :1 logistic1
tree :1 logistic1
tree :0 logistic0
tree :1 logistic1
tree :1 logistic1
tree :1 logistic1
tree :1 logistic1
tree :1 logistic1
tree :0 logistic0
tree :1 logistic1
tree :1 logistic1
tree :1 logistic1
tree :1 logistic1
tree :1 logistic1
tree :1 logistic1
tree :1 logistic1
tree :1 logistic1
tree :1 logistic1
tree :1 logistic1
tree :1 logistic1
tree :0 logistic0
tree :1 logistic1
tree :1 logistic1
tree :1 logistic1
tree :1 logistic1
tree :1 logistic1
tree :1 logistic1
tree :1 logistic1
tree :1 logistic1
tree :1 logistic1
tree :0 logistic0
tree :1 logistic1
tree :1 logistic1
tree :1 logistic1
tree :1 logistic1
tree :1 logistic1
tree :1 logistic1
tree :1 logistic1
tree :1 logistic1
tree :1 logistic1
tree :1 logistic1
tree :1 logistic1
tree :1 logistic1
tree :1 logistic1
tree :1 logistic1
tree :1 logistic1
tree :1 logistic1
tree :1 logistic1
tree :1 logistic1
tree :1 logistic1
tree :0 lo